In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)
import numpy as np
import statistics
from pprint import pprint
from math import isnan

sp500_history = pd.read_excel('./sp500_rebalance_announcements.xlsx')
sp500_history['effective_date'] = pd.to_datetime(sp500_history['effective_date']) 
sp500_history['announcement_date'] = pd.to_datetime(sp500_history['announcement_date']) 

start_date = datetime.strptime('2016-1-1', '%Y-%m-%d')
end_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history['effective_date'] > start_date) & (sp500_history['effective_date'] <= end_date)
sp500_history = sp500_history.loc[mask]

# Sort
sp500_history = sp500_history.sort_values(['announcement_date', 'type'], ascending=[True, True])
sp500_history

,Unnamed: 0,type,name,ticker,announcement_date,effective_date
42,42,ADDED,Willis Towers Watson,WSH,2015-12-28,2016-01-04
43,43,DELETED,Fossil Group,FOSL,2015-12-28,2016-01-04
44,44,ADDED,Extra Space Storage,EXR,2016-01-13,2016-01-15
45,45,DELETED,Chubb,CB,2016-01-13,2016-01-15
46,46,ADDED,Federal Realty Trust,FRT,2016-01-22,2016-01-29
...,...,...,...,...,...,...
17,17,DELETED,Cimarex Energy,XEC,2020-02-27,2020-03-03
18,18,ADDED,Otis Worldwide,OTIS,2020-03-31,2020-04-03
19,19,ADDED,Carrier Global,CARR,2020-03-31,2020-04-03
20,20,DELETED,Raytheon,RTN,2020-03-31,2020-04-06


In [6]:
calendar = pd.read_excel('./Nasdaq_Trading_Calendar.xlsx', sheet_name=None)
rebal_dates = []
for year in calendar:
    sheet = calendar[year]
    mask = sheet['S&P Indexes Rebalance S&P 500, S&P 400, and S&P 600'] == 1
    year_rebal_dates = sheet.loc[mask]
    for date in year_rebal_dates['Date'].values:
        rebal_dates.append(date)
pprint(rebal_dates)

[numpy.datetime64('2020-03-20T00:00:00.000000000'),
 numpy.datetime64('2020-06-19T00:00:00.000000000'),
 numpy.datetime64('2020-09-18T00:00:00.000000000'),
 numpy.datetime64('2020-12-18T00:00:00.000000000'),
 numpy.datetime64('2019-03-15T00:00:00.000000000'),
 numpy.datetime64('2019-06-21T00:00:00.000000000'),
 numpy.datetime64('2019-09-20T00:00:00.000000000'),
 numpy.datetime64('2019-12-20T00:00:00.000000000'),
 numpy.datetime64('2018-03-16T00:00:00.000000000'),
 numpy.datetime64('2018-06-15T00:00:00.000000000'),
 numpy.datetime64('2018-09-21T00:00:00.000000000'),
 numpy.datetime64('2018-12-21T00:00:00.000000000'),
 numpy.datetime64('2017-03-17T00:00:00.000000000'),
 numpy.datetime64('2017-06-16T00:00:00.000000000'),
 numpy.datetime64('2017-09-15T00:00:00.000000000'),
 numpy.datetime64('2017-12-15T00:00:00.000000000'),
 numpy.datetime64('2016-03-18T00:00:00.000000000'),
 numpy.datetime64('2016-06-17T00:00:00.000000000'),
 numpy.datetime64('2016-09-16T00:00:00.000000000'),
 numpy.datet

In [7]:
# rebal_type is 'regular' or 'ad_hoc'
# add_delete is 'add' or 'delete'
# entry_date and exit_date are in terms of number of days before (-) or after (+) effective date
# entry_time and exit_time are 'Open' or 'Close'
strategy_attributes = ['rebal_type', 'add_delete', 'entry_date', 'entry_time', 'exit_date', 'exit_time']

impl_output_columns = ['implementation_date', 'effective_date']
impl_output_columns.extend(strategy_attributes)
impl_output_columns.extend(['total', 'up', 'count'])

impl_output = pd.DataFrame(columns=impl_output_columns)

ann_output_columns = ['announcement_date']
ann_output_columns.extend(strategy_attributes)
ann_output_columns.extend(['total', 'up', 'count'])

ann_output = pd.DataFrame(columns=ann_output_columns)